<a href="https://colab.research.google.com/github/adrian-lison/gnn-community-detection/blob/master/Notebooks/LGNN_Semi_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model #4 LGNN
----------


In [85]:
#!pip install dgl

## Imports

In [86]:
# dgl
import dgl
import dgl.function as fn
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh

# pytorch
import torch as th
import torch.nn as nn
import torch.nn.functional as F

# other
import time
import numpy as np
import random as rng
import scipy.sparse as ss
import networkx as nx
import pickle

## GNN Definition

In [87]:
# Return a list containing features gathered from multiple radius.
import dgl.function as fn
def aggregate_radius(radius, g, z):
    # initializing list to collect message passing result
    z_list = []
    g.ndata['z'] = z
    # pulling message from 1-hop neighbourhood
    g.update_all(fn.copy_src(src='z', out='m'), fn.sum(msg='m', out='z'))
    z_list.append(g.ndata['z'])
    for i in range(radius - 1):
        for j in range(2 ** i):
            #pulling message from 2^j neighborhood
            g.update_all(fn.copy_src(src='z', out='m'), fn.sum(msg='m', out='z'))
        z_list.append(g.ndata['z'])
    return z_list

In [88]:
class LGNNCore(nn.Module):
    def __init__(self, in_feats, out_feats, radius):
        super(LGNNCore, self).__init__()
        self.out_feats = out_feats
        self.radius = radius

        self.linear_prev = nn.Linear(in_feats, out_feats)
        self.linear_deg = nn.Linear(in_feats, out_feats)
        self.linear_radius = nn.ModuleList(
                [nn.Linear(in_feats, out_feats) for i in range(radius)])
        self.linear_fuse = nn.Linear(in_feats, out_feats)
        self.bn = nn.BatchNorm1d(out_feats)

    def forward(self, g, feat_a, feat_b, deg, pm_pd):
        # term "prev"
        prev_proj = self.linear_prev(feat_a)
        # term "deg"
        deg_proj = self.linear_deg(deg * feat_a)

        # term "radius"
        # aggregate 2^j-hop features
        hop2j_list = aggregate_radius(self.radius, g, feat_a)
        # apply linear transformation
        hop2j_list = [linear(x) for linear, x in zip(self.linear_radius, hop2j_list)]
        radius_proj = sum(hop2j_list)

        # term "fuse"
        fuse = self.linear_fuse(th.mm(pm_pd, feat_b))

        # sum them together
        result = prev_proj + deg_proj + radius_proj + fuse

        # skip connection and batch norm
        n = self.out_feats // 2
        result = th.cat([result[:, :n], F.relu(result[:, n:])], 1)
        result = self.bn(result)

        return result

In [89]:
class LGNNLayer(nn.Module):
    def __init__(self, in_feats, out_feats, radius):
        super(LGNNLayer, self).__init__()
        self.g_layer = LGNNCore(in_feats, out_feats, radius)
        self.lg_layer = LGNNCore(in_feats, out_feats, radius)

    def forward(self, g, lg, x, lg_x, deg_g, deg_lg, pm_pd):
        next_x = self.g_layer(g, x, lg_x, deg_g, pm_pd)
        pm_pd_y = th.transpose(pm_pd, 0, 1)
        next_lg_x = self.lg_layer(lg, lg_x, x, deg_lg, pm_pd_y)
        return next_x, next_lg_x

In [90]:
class LGNN(nn.Module):
    def __init__(self, radius):
        super(LGNN, self).__init__()
        self.layer1 = LGNNLayer(1, 16, radius)  # input is scalar feature
        self.layer2 = LGNNLayer(16, 16, radius)  # hidden size is 16
        self.layer3 = LGNNLayer(16, 16, radius)
        self.linear = nn.Linear(16, 7)  # predice seven classes

    def forward(self, g, lg, pm_pd):
        # compute the degrees
        deg_g = g.in_degrees().float().unsqueeze(1)
        deg_lg = lg.in_degrees().float().unsqueeze(1)
        # use degree as the input feature
        x, lg_x = deg_g, deg_lg
        x, lg_x = self.layer1(g, lg, x, lg_x, deg_g, deg_lg, pm_pd)
        x, lg_x = self.layer2(g, lg, x, lg_x, deg_g, deg_lg, pm_pd)
        x, lg_x = self.layer3(g, lg, x, lg_x, deg_g, deg_lg, pm_pd)
        return self.linear(x)

## Data Loading

In [91]:
#Loading CORA
data = citegrh.load_cora()
features = th.FloatTensor(data.features)
labels = th.LongTensor(data.labels)
mask = th.ByteTensor(data.train_mask)
g = data.graph

In [92]:
matrix = lil_matrix((data.graph.number_of_nodes(),data.graph.number_of_edges()))
for e in data.graph.edges:
    matrix[e[0],e[1]] = 1
inputs_pmpd = ss.coo_matrix(matrix,dtype="int64")
inputs_pmpd

<2708x10556 sparse matrix of type '<class 'numpy.int64'>'
	with 10556 stored elements in COOrdinate format>

In [93]:
g = DGLGraph(g) # turn networkx graph into DGL graph

In [94]:
print('We have %d nodes.' % g.number_of_nodes())
print('We have %d edges.' % g.number_of_edges())

We have 2708 nodes.
We have 10556 edges.


## Select Training Set

In [95]:
percentage_train = 0.02

with open("data/cora_permutation1.pickle","rb") as f:
    perm1 = pickle.load(f)
mask = np.zeros(g.number_of_nodes())
mask[perm1[range(int(percentage_train*g.number_of_nodes()))]] = 1
mask = th.ByteTensor(mask)

## Training

In [99]:
# create the model
net = LGNN(radius=3)
# define the optimizer
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)

# a util function to convert a scipy.coo_matrix to torch.SparseFloat
def sparse2th(mat):
    value = mat.data
    indices = th.LongTensor([mat.row, mat.col])
    tensor = th.sparse.FloatTensor(indices, th.from_numpy(value).float(), mat.shape)
    return tensor

all_logits = []

pmpd = sparse2th(inputs_pmpd)
lg = g.line_graph(backtracking=False)

# train
for epoch in range(100):

    # Compute loss for test nodes (only for validation, not used by optimizer)
    net.eval()
    prediction = F.log_softmax(net(g, lg, pmpd),1)
    val_loss = F.nll_loss(prediction.detach()[1-mask], labels[1-mask])
    net.train()

    logits = net(g, lg, pmpd)
    # Save logits for visualization later
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)

    # Compute loss for train nodes
    loss = F.nll_loss(logp[mask], labels[mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('Epoch %d | Loss: %.4f | Total: %.4f' % (epoch, loss.item(), val_loss.item()))

Epoch 0 | Loss: 1.9922 | Total: 274970443776.0000
Epoch 1 | Loss: 1.8936 | Total: 7.5828
Epoch 2 | Loss: 1.7970 | Total: 1.9283
Epoch 3 | Loss: 1.7094 | Total: 1.8867
Epoch 4 | Loss: 1.6568 | Total: 1.8358
Epoch 5 | Loss: 1.6377 | Total: 93.5670
Epoch 6 | Loss: 1.5826 | Total: 14.2129
Epoch 7 | Loss: 1.5720 | Total: 3.4183
Epoch 8 | Loss: 1.5470 | Total: 1.7551
Epoch 9 | Loss: 1.5259 | Total: 1.7038
Epoch 10 | Loss: 1.5019 | Total: 1.7021
Epoch 11 | Loss: 1.4786 | Total: 1.7034
Epoch 12 | Loss: 1.5199 | Total: 1.6926
Epoch 13 | Loss: 1.4528 | Total: 1.6937
Epoch 14 | Loss: 1.4353 | Total: 1.6920
Epoch 15 | Loss: 1.4235 | Total: 1.6880
Epoch 16 | Loss: 1.4099 | Total: 1.6841
Epoch 17 | Loss: 1.4002 | Total: 1.6812
Epoch 18 | Loss: 1.3887 | Total: 1.6744
Epoch 19 | Loss: 1.3782 | Total: 1.6710
Epoch 20 | Loss: 1.3683 | Total: 1.6723
Epoch 21 | Loss: 1.3572 | Total: 1.6698
Epoch 22 | Loss: 1.3458 | Total: 1.6751
Epoch 23 | Loss: 1.3348 | Total: 1.6823
Epoch 24 | Loss: 1.3216 | Total: 1.69

## Evaluation

In [97]:
from sklearn.metrics import accuracy_score as acc
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = F.log_softmax(net(g, lg, pmpd),1).detach()

pred_sets = {"All ":final_prediction,"Train":final_prediction[mask],"Test":final_prediction[1-mask]}
label_sets = {"All ":labels,"Train":labels[mask],"Test":labels[1-mask]}
eval_functions = {"NLL-Loss":lambda y,x: F.nll_loss(x,y),"Accuracy":lambda y,x: acc(y,x.numpy().argmax(axis=1))}

for name,func in eval_functions.items():
    eval_message = f"\n{name}:\n"
    for subset in pred_sets.keys():
        eval_message += f" {subset}: {func(label_sets[subset],pred_sets[subset]):.4f} |"
    print(eval_message)


NLL-Loss:
 All : 1.9709 | Train: 0.9894 | Test: 1.9909 |

Accuracy:
 All : 0.4140 | Train: 0.6111 | Test: 0.4099 |
